In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [9]:
import os

file_paths = [
    "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-1.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-2.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-3.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-4.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-5.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-6.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-7.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-8.txt",
#     "/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-9.txt"
]

text = ""

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        text += file.read()


In [4]:
lines = text.split('\n')

len(lines)

7547845

In [5]:
len(text)

478110829

In [10]:
!pip install hazm
!pip install num2fawords
!pip install parsinorm

In [11]:
import re
from hazm import Normalizer, InformalNormalizer, word_tokenize
from num2fawords import words
from parsinorm import General_normalization, Date_time_to_text, Abbreviation
from string import punctuation

def normalize(text):
    hazm = Normalizer()
    text = hazm.normalize(text)
#     print("1", text)
#     text = re.sub('''[!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~؟،٪×÷»«]+''', ' ', text)
#     text = ' '.join(text.split())
    text = text.replace('ء', '')
    text = text.replace('أ', 'ا')
    text = text.replace('ؤ', 'و')
    text = text.replace('ئ', 'ی')
    pattern = r'[^آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی \u200C]'
    text = re.sub(pattern, '', text)
#     print("2", text)
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if not any(char.isalpha() and ord(char) < 128 for char in token)]
    text = ' '.join(filtered_tokens)
    return text

In [12]:
sentences = text.splitlines()

for sentence in sentences[:5]:
    print("1", sentence)

1 عنوان مقاله: صفحهٔ اصلی
1 
1 <templatestyles src="صفحه اصلی/
1 
1 


In [54]:
len(sentences)

7547847

In [13]:
from tqdm import tqdm
cleaned_sentences = [sentence for sentence in tqdm(sentences) if sentence != ""]


100%|██████████| 446023/446023 [00:00<00:00, 1922294.70it/s]


In [56]:
len(cleaned_sentences)

2882884

In [57]:
for sentence in cleaned_sentences[:5]:
    print("1", sentence)

1 عنوان مقاله: صفحهٔ اصلی
1 <templatestyles src="صفحه اصلی/
1 عنوان مقاله: ویکی پدیا
1 ویکی پدیا (کوته نوشت به صورت «وپ» و «WP») یک دانشنامه برخط چندزبانه مبتنی بر وب با محتوای آزاد و همکاری باز است که با همکاری افراد داوطلب نوشته می شود و هر کسی که به اینترنت و وب دسترسی داشته باشد می تواند مقالات آن را ببیند و ویرایش کند. نام ویکی پدیا واژه ای ترکیبی است که از واژه های ویکی (وبگاه مشارکتی) و اِنسایکلوپدیا (Encyclopedia) (دانشنامه یا دائرةالمعارف) گرفته شده است. هدف ویکی پدیا آفرینش و انتشار جهانی یک دانشنامه با محتوای آزاد به تمامی زبان های زندهٔ دنیا است.
1 ویکی پدیای انگلیسی در تاریخ ۱۵ ژانویه ۲۰۰۱ (۲۶ دی ۱۳۷۹) به صورت مکملی برای دانشنامهٔ تخصصی نیوپدیا نوشته شد. بنیان گذاران آن «جیمی ویلز» و «لری سنگر» هستند. هم اکنون بنیاد غیرانتفاعی ویکی مدیا پروژهٔ ویکی پدیا را پشتیبانی می کند. میزبان های اینترنتی اصلی این وبگاه در شهر تامپای فلوریدا هستند. همچنین میزبان های اضافی دیگری هم در شهرهای آمستردام و سئول به این وبگاه یاری می رسانند.


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĩĪīĭĮįİıĴĵĶķĸĺĻļĽľŁłŃńŅņŇňŋŌōŏőŒœŔŕŗŘřŚśŜŝŞşŠšţŤťŦŧŨũŪūŬŭŮůűųŷŹźŻżŽžƀƁƂƃƇƈƉƊƋƌƎƏƑƒƓƕƗƘƙƜƝƞƟƠơƢƣƤƥƩƬƭƮƯưƲƳƴƵƶƿǀǁǃǍǎǏǐǒǔǖǚǜǝǟǡǣǤǥǦǧǨǪǫǭǰǴǵǸǹǽȊȋȒȓȘșȚțȜȝȞȟȠȤȥȦȧȲȳȷȻȼɀɃɄɈɉɊɋɌɍɎɏɐɑɒɓɔɕɖɗɘəɚɛɜɟɠɡɢɣɤɥɦɨɪɫɬɯɰɱɲɳɴɵɸɹɺɻɽɾʀʁʂʃʄʈʉʊʋʌʍʎʏʐʑʒʔʕʘʙʛʝʠʰʱʲʷʹʺʻʼʽʾʿˀˁ˂˃˄ˆˇˈˉˊˌˏːˑ˓˘˚˛˝ˠˢˤˮ̵̶̷̸̡̢̧̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿͂͆͊͋͌̚ͅ͏͓͔͕͖͙͚͐͑͒͗͛ͣͤͥͦͧͨͩͪͫͬͭͮͯ͘͜͟͢͝͠͡Ͱͱ΄ΆΈΉΊΌΎΏΐΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩάέήίαβγδεζηθικλμνξοπρςστυφχψωϊόύώϐϑϒϕϖϚϛϜϝϞϟϠϡϣϥϩϬϵϷϸϺϻЀЁЂЃЄІЇЈЉЊЋЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяѐёђѓєіїјљњћѝўџѢѣѦѧѩѫѲѳѷ҉ҐґҒғҖҗҘҙҚқҠҡҢңҤҥҪҫҮүҰұҲҳҶҷҹҺһӊӑӓӕӖӗӘәӣӦӧӨөӮӯӰӱӳԑԱԲԳԴԵԶԷԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇՈՉՊՋՌՍՎՏՑՒՓՔՕՖաբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆևְֱֲֳִֵֶַָֹֻּ֗֫־ֿ׀ׁׂאבגדהוזחטיךכלםמןנסעףפץצקרשתװײ׳،ؐؓ؛؜؟ءآأؤإئابةتثجح

In [14]:
from tqdm.auto import tqdm
normalized_sentences = [normalize(sentence) for sentence in tqdm(cleaned_sentences, desc="Normalizing sentences")]

Normalizing sentences:   0%|          | 0/212722 [00:00<?, ?it/s]

In [8]:
for sentence in normalized_sentences[:10]:
    print("1", sentence)

1 عنوان مقاله صفحه اصلی
1 صفحه اصلی
1 عنوان مقاله ویکی پدیا
1 ویکی پدیا کوته نوشت به صورت وپ و یک دانشنامه برخط چندزبانه مبتنی بر وب با محتوای آزاد و همکاری باز است که با همکاری افراد داوطلب نوشته_می‌شود و هر کسی که به اینترنت و وب دسترسی داشته_باشد می‌تواند مقالات آن را ببیند و ویرایش کند نام ویکی پدیا واژه‌ای ترکیبی است که از واژه‌های ویکی وبگاه مشارکتی و انسایکلوپدیا دانشنامه یا دایرالمعارف گرفته_شده_است هدف ویکی پدیا آفرینش و انتشار جهانی یک دانشنامه با محتوای آزاد به تمامی زبان‌های زنده دنیا است
1 ویکی پدیای انگلیسی در تاریخ ژانویه دی به صورت مکملی برای دانشنامه تخصصی نیوپدیا نوشته_شد بنیان گذاران آن جیمی ویلز و لری سنگر هستند هم اکنون بنیاد غیرانتفاعی ویکی مدیا پروژه ویکی پدیا را پشتیبانی می‌کند میزبان‌های اینترنتی اصلی این وبگاه در شهر تامپای فلوریدا هستند همچنین میزبان‌های اضافی دیگری هم در شهرهای آمستردام و سیول به این وبگاه یاری می‌رسانند
1 ویکی پدیا از پایان آوریل تا اکتبر یکی از وبگاه برتر جهان از لحاظ شمار بازدیدکنندگان بوده_است که بیش از نیمی از بازدیدها به ویکی پدیای انگ

In [61]:
res = normalize("سلام تست میکنیم /")
print(res)

سلام تست میکنیم


In [15]:
combined_sentences = ''.join(normalized_sentences)
chars = sorted(list(set(combined_sentences)))
print(chars)

[' ', '_', 'آ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c']


In [16]:
sentences = [sentence.replace('_', '') for sentence in normalized_sentences]

In [11]:
combined_sentences = ''.join(sentences)
chars = sorted(list(set(combined_sentences)))
print(chars)

[' ', 'آ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c']


In [12]:
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 آابتثجحخدذرزسشصضطظعغفقلمنهوپچژکگی‌
35


In [17]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

In [18]:
data = torch.tensor(encode(combined_sentences), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
with open('combined_sentences.txt', 'w', encoding='utf-8') as f:
    for line in combined_sentences:
        f.write(line + '\n')

In [22]:
from torch.utils.data import DataLoader, Dataset
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data) - block_size

    def __getitem__(self, idx):
        x = self.data[idx:idx + block_size]
        y = self.data[idx + 1:idx + block_size + 1]
        return x, y

train_dataset = TextDataset(train_data)
val_dataset = TextDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split, loader in [('train', train_loader), ('val', val_loader)]:
        losses = torch.zeros(eval_iters)
        for k, (X, Y) in enumerate(loader):
            if k >= eval_iters:
                break
            X, Y = X.to(device), Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [33]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [26]:
from torch import nn

In [27]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  
        q = self.query(x) 
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [28]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [29]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [30]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [31]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [32]:
def calculate_perplexity(loss):
    return torch.exp(loss)

In [43]:
import torch.nn as nn
from torch.nn import functional as F
batch_size = 10
block_size = 256 
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [ ]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_perplexity = calculate_perplexity(losses['train'])
        val_perplexity = calculate_perplexity(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f} (perplexity {train_perplexity:.4f}), val loss {losses['val']:.4f} (perplexity {val_perplexity:.4f})")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

In [43]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_perplexity = calculate_perplexity(losses['train'])
        val_perplexity = calculate_perplexity(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f} (perplexity {train_perplexity:.4f}), val loss {losses['val']:.4f} (perplexity {val_perplexity:.4f})")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

10.765859 M parameters
step 0: train loss 3.6594 (perplexity 38.8378), val loss 3.6576 (perplexity 38.7687)
step 250: train loss 2.5578 (perplexity 12.9077), val loss 2.5653 (perplexity 13.0052)
step 500: train loss 2.5383 (perplexity 12.6576), val loss 2.5465 (perplexity 12.7627)
step 750: train loss 2.5067 (perplexity 12.2640), val loss 2.5143 (perplexity 12.3581)
step 1000: train loss 2.2767 (perplexity 9.7443), val loss 2.2870 (perplexity 9.8458)
step 1250: train loss 2.1237 (perplexity 8.3622), val loss 2.1271 (perplexity 8.3902)
step 1500: train loss 2.0280 (perplexity 7.5990), val loss 2.0314 (perplexity 7.6251)
step 1750: train loss 1.9458 (perplexity 6.9990), val loss 1.9594 (perplexity 7.0948)
step 2000: train loss 1.8878 (perplexity 6.6046), val loss 1.9036 (perplexity 6.7101)
step 2250: train loss 1.8258 (perplexity 6.2078), val loss 1.8408 (perplexity 6.3013)
step 2500: train loss 1.7809 (perplexity 5.9352), val loss 1.7900 (perplexity 5.9896)
step 2750: train loss 1.7259 

In [44]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_perplexity = calculate_perplexity(losses['train'])
        val_perplexity = calculate_perplexity(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f} (perplexity {train_perplexity:.4f}), val loss {losses['val']:.4f} (perplexity {val_perplexity:.4f})")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 1.5098 (perplexity 4.5257), val loss 1.5023 (perplexity 4.4921)
step 250: train loss 1.4996 (perplexity 4.4800), val loss 1.5138 (perplexity 4.5439)
step 500: train loss 1.4893 (perplexity 4.4340), val loss 1.4945 (perplexity 4.4571)
step 750: train loss 1.4614 (perplexity 4.3121), val loss 1.4780 (perplexity 4.3842)
step 1000: train loss 1.4654 (perplexity 4.3292), val loss 1.4803 (perplexity 4.3941)
step 1250: train loss 1.4536 (perplexity 4.2785), val loss 1.4544 (perplexity 4.2820)
step 1500: train loss 1.4461 (perplexity 4.2463), val loss 1.4604 (perplexity 4.3078)
step 1750: train loss 1.4321 (perplexity 4.1875), val loss 1.4472 (perplexity 4.2513)
step 2000: train loss 1.4264 (perplexity 4.1636), val loss 1.4407 (perplexity 4.2236)
step 2250: train loss 1.4372 (perplexity 4.2087), val loss 1.4264 (perplexity 4.1638)
step 2500: train loss 1.4228 (perplexity 4.1489), val loss 1.4270 (perplexity 4.1661)
step 2750: train loss 1.4185 (perplexity 4.1308), val loss 1

In [44]:
# create a PyTorch optimizer
import random
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_perplexity = calculate_perplexity(losses['train'])
        val_perplexity = calculate_perplexity(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f} (perplexity {train_perplexity:.4f}), val loss {losses['val']:.4f} (perplexity {val_perplexity:.4f})")



    batch_idx = random.randint(0, len(train_loader) - 1)
    X, Y = train_loader.dataset[batch_idx]
    X, Y = X.to(device), Y.to(device)

    logits, loss = model(X.unsqueeze(0), Y.unsqueeze(0))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


10.765859 M parameters
step 0: train loss 3.6221 (perplexity 37.4142), val loss 3.6188 (perplexity 37.2946)
step 100: train loss 2.7155 (perplexity 15.1118), val loss 2.7087 (perplexity 15.0102)
step 200: train loss 2.6508 (perplexity 14.1655), val loss 2.6383 (perplexity 13.9887)
step 300: train loss 2.6246 (perplexity 13.7996), val loss 2.6179 (perplexity 13.7063)
step 400: train loss 2.6132 (perplexity 13.6429), val loss 2.5973 (perplexity 13.4279)
step 500: train loss 2.6136 (perplexity 13.6479), val loss 2.6115 (perplexity 13.6196)
step 600: train loss 2.5999 (perplexity 13.4621), val loss 2.5869 (perplexity 13.2880)
step 700: train loss 2.5958 (perplexity 13.4077), val loss 2.5959 (perplexity 13.4092)
step 800: train loss 2.5978 (perplexity 13.4339), val loss 2.5970 (perplexity 13.4233)
step 900: train loss 2.5934 (perplexity 13.3753), val loss 2.5874 (perplexity 13.2956)
step 1000: train loss 2.5897 (perplexity 13.3251), val loss 2.5891 (perplexity 13.3174)
step 1100: train loss

In [46]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained('flax-community/gpt2-medium-persian')
model = GPT2LMHeadModel.from_pretrained('flax-community/gpt2-medium-persian')

dataset = load_dataset('text', data_files={'train': '/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-1.txt', 'validation': '/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-9.txt'})

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare DataLoader
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train the model
trainer.train()

Map:   0%|          | 0/446023 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('flax-community/gpt2-medium-persian')
model = GPT2LMHeadModel.from_pretrained('flax-community/gpt2-medium-persian')

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the data
tokens = tokenizer(combined_sentences, return_tensors='pt', max_length=256, truncation=True, padding='max_length')
data = tokens['input_ids'].squeeze()  # Flatten the tensor

# Split the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

block_size = 256  # Adjust as necessary
train_dataset = TextDataset(train_data, block_size)
val_dataset = TextDataset(val_data, block_size)

# Prepare DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(val_dataset, batch_size=4)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # reduced batch size
    per_device_eval_batch_size=4,  # reduced batch size
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,  
    fp16=True,# accumulate gradients over 2 steps
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


In [ ]:
from transformers import (
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
)


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer, file_path=train_path, block_size=256
    )

    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    return train_dataset, test_dataset, data_collator


# Freezing the lower layers increases the training speed and reduces the memory requirement.
# Depending on your task you may want to freeze all layers and train addition layers that you are adding to the model
# or unfreeze as many layers that you can affort training with a reasonable batchsize.
def freeze_lower_layers():
    for param in model.base_model.parameters():
        param.requires_grad = False

    for param in (
        model.base_model.h[23].parameters() or model.base_model.h[22].parameters()
    ):
        param.requires_grad = True


# load model
model = AutoModelWithLMHead.from_pretrained("bolbolzaban/gpt2-persian")

# freeze lower layers and only train top layers
freeze_lower_layers()

# load dataset
tokenizer = AutoTokenizer.from_pretrained("bolbolzaban/gpt2-persian")
train_dataset, test_dataset, data_collator = load_dataset(
    "/kaggle/working/combined_sentences.txt", "/kaggle/working/combined_sentences.txt", tokenizer
)

# train
training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    # Set the batch size to a maximum value that could fit into GPU memory,
    # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    eval_steps=1000,
    save_steps=1000,
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# save
trainer.save_model()

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1712: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.138700
1000,1.878500
1500,1.835200
2000,1.812000
2500,1.792400
3000,1.783800
3500,1.772100
4000,1.763400
4500,1.754800
5000,1.748800
